In [52]:
from datetime import datetime

import pandas as pd
import numpy as np
import plotly.graph_objects as go

np.set_printoptions(suppress=True, precision=5) # Disable scientific notation

SELECTED_RUNS = [
    68, 69, 70, 71, 72, 73
]
k_y = 1.059952828992417 # see notion for derivation

In [84]:
def get_run_data(selected_runs):
    metric_data = pd.DataFrame()

    for rn in selected_runs:
        # Import the data
        metric_datum = pd.read_csv(f'./SN5_Round9_DriveBrake/run{rn}.dat')

        metric_data = pd.concat([metric_data, metric_datum])
        print(f"Loaded file run{rn}.dat ({len(metric_datum)} rows)")
    
    # Rescale forces for tire diam
    # metric_data["FX (unscaled)"] = metric_data["FX"]
    # metric_data["FX"] = k_y * metric_data["FX"]

    return metric_data

In [110]:
def format_figure(fig, title, xaxis_title, yaxis_title, ):
    fig.update_layout(
        title_text=title,
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        legend=dict(
            # x=1.02, y=1,
            bgcolor='rgba(255,255,255,0.7)',
            bordercolor='black',
            borderwidth=1,
            title="Legend"
        ),
        template="plotly_white"
    )

In [71]:
def scatter_long_over_normal(data):
    data_sample = data.sample(n=10000)
    return go.Scatter(
        x=data_sample["FZ"], 
        y=data_sample["FX"],
        mode="markers",
        marker=dict(
            color=data["SA"], 
            colorscale="Viridis", 
            showscale=True, 
            colorbar=dict(title="Slip Angle (deg)")
        ),
        name="Sampled Data"
    )

In [73]:
def scatter_abs_long(data):
    data_sample = data.sample(n=10000)
    return go.Scatter(
        x=data_sample["FZ"], 
        y=abs(data_sample["FX"]),
        mode="markers",
        marker=dict(
            color=data["SA"], 
            colorscale="Viridis", 
            showscale=True, 
            colorbar=dict(title="Slip Angle (deg)")
        ),
        name="Sampled Data"
    )

In [112]:
def scatter_abs_long_with_ratio(data):
    # data_sample = data.sample(n=10000)
    
    absolute_long = abs(data["FX"])
    absolute_load = abs(data["FZ"])

    long_load_ratio = absolute_long / absolute_load
    
    return go.Scatter(
        x=data["FZ"], 
        y=absolute_long,
        mode="markers",
        # marker=dict(
        #     color=long_load_ratio, 
        #     colorscale="Viridis", 
        #     showscale=True, 
        #     colorbar=dict(title="|Fx / Fz|")
        # ),
        name="All Data"
    )

In [115]:
def scatter_linear(xrange, slope):
    x = np.linspace(xrange[0], xrange[1], 1000)
    return go.Scatter(
        x=x, 
        y=slope * x,
        name=f"y = {round(slope, 2)}x"
    )

In [76]:
def find_friction_coefficient(data):
    absolute_long = abs(data["FX"])
    absolute_load = abs(data["FZ"])

    long_load_ratio = absolute_long / absolute_load

    return max(long_load_ratio)

In [95]:
def fig_friction_coeff_SA(data):
    data_sample = data.sample(1000)
    
    absolute_long = abs(data_sample["FX"])
    absolute_load = abs(data_sample["FZ"])

    long_load_ratio = absolute_long / absolute_load
    
    return go.Scatter(
        x=data_sample["SA"], 
        y=long_load_ratio,
        mode="markers",
        # marker=dict(
        #     color=long_load_ratio, 
        #     colorscale="Viridis", 
        #     showscale=True, 
        #     colorbar=dict(title="|Fx / Fz|")
        # ),
        name="Sampled Data"
    )

In [85]:
all_data = get_run_data(SELECTED_RUNS)

Loaded file run68.dat (19343 rows)
Loaded file run69.dat (76297 rows)
Loaded file run70.dat (67007 rows)
Loaded file run71.dat (19343 rows)
Loaded file run72.dat (75860 rows)
Loaded file run73.dat (66778 rows)


In [72]:
timestamp = datetime.now().strftime("%m%d_%H%M%S")

fig = go.Figure()
fig.add_trace(scatter_long_over_normal(all_data))
format_figure(fig, "Longitudinal Force over Load Force", "Load Force (N)", "Longitudinal Force (N)")

fig.write_html(f"fx_out/figures/longitudinal_over_load_{timestamp}.html")

In [75]:
timestamp = datetime.now().strftime("%m%d_%H%M%S")

fig = go.Figure()
fig.add_trace(scatter_abs_long(all_data))
format_figure(fig, "Absolute Longitudinal Force over Load Force", "Load Force (N)", "Absolute Longitudinal Force (N)")

fig.write_html(f"fx_out/figures/abs_long_over_load{timestamp}.html")

In [116]:
timestamp = datetime.now().strftime("%m%d_%H%M%S")

fig = go.Figure()
fig.add_trace(scatter_abs_long_with_ratio(all_data[(abs(all_data["SA"]) < 1) & (all_data["FZ"] < -500) & (all_data["FZ"] > -1300)]))
fig.add_trace(scatter_linear((-1300, 0), -3.1700277852469303))
format_figure(fig, "Absolute Longitudinal Force over Load Force at 0 slip angle", "Load Force (N)", "Absolute Longitudinal Force (N)")

fig.update_layout(xaxis=dict(range=[-1300, 0]))

fig.write_html(f"fx_out/figures/abs_long_load_ratios{timestamp}.html")

In [96]:
timestamp = datetime.now().strftime("%m%d_%H%M%S")

fig = go.Figure()
fig.add_trace(fig_friction_coeff_SA(all_data))
format_figure(fig, "Force Ratio over Slip Angle", "Slip Angle (deg)", "Absolute Longitudinal Force / Absolute Load Force")

fig.write_html(f"fx_out/figures/ratio_over_slip_{timestamp}.html")

In [86]:
find_friction_coefficient(all_data[all_data["FZ"] < -500])

3.1700277852469303
